<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/3_5_10_RAG_PDF_Loader_%E1%84%87%E1%85%B5%E1%84%80%E1%85%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 미션 14: RAG 시스템 구현

## Part 1: PDF 로더 비교 - PyPDF vs PyMuPDF

RAG 시스템의 첫 번째 단계는 **문서 로드**입니다. PDF 문서를 텍스트로 변환하는 방식에 따라 후속 처리 품질이 달라집니다.

이번 실습에서는 LangChain에서 제공하는 두 가지 PDF 로더를 비교합니다:

| 라이브러리 | LangChain 클래스 | 특징 |
|-----------|-----------------|------|
| **PyPDF** | `PyPDFLoader` | 순수 Python, 가벼움, 기본적인 텍스트 추출 |
| **PyMuPDF** | `PyMuPDFLoader` | C 기반 바인딩, 빠름, 레이아웃 보존 우수 |

---

## 1️⃣ 환경 설정

In [ ]:
# 필요한 라이브러리 설치
!pip install -q pypdf pymupdf langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
# 라이브러리 임포트
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
import time

# PDF 파일 경로 (Colab에서는 파일 업로드 후 경로 지정)
PDF_PATH = "2024년_원천징수의무자를_위한_연말정산_신고안내.pdf"

# 비교할 페이지 번호 (1-indexed)
PAGE_NUMBERS = [8, 84]

In [ ]:
# 파일 업로드
from google.colab import files
uploaded = files.upload()

Saving 2024년_원천징수의무자를_위한_연말정산_신고안내.pdf to 2024년_원천징수의무자를_위한_연말정산_신고안내.pdf


---

## 2️⃣ PyPDFLoader로 문서 로드

### PyPDFLoader 특징
- **순수 Python** 구현으로 추가 시스템 의존성 없음
- 각 페이지를 개별 `Document` 객체로 반환
- 메타데이터에 `source`, `page` 정보 포함

In [ ]:
# PyPDFLoader 사용
print("="*60)
print("📄 PyPDFLoader 로드 시작")
print("="*60)

PDF_PATH = list(uploaded.keys())[0]

start_time = time.time()
pypdf_loader = PyPDFLoader(PDF_PATH)
pypdf_docs = pypdf_loader.load()
pypdf_time = time.time() - start_time

print(f"\n 로드 완료!")
print(f"총 페이지 수: {len(pypdf_docs)}")
print(f"소요 시간: {pypdf_time:.2f}초")

📄 PyPDFLoader 로드 시작

 로드 완료!
총 페이지 수: 426
소요 시간: 61.35초


In [ ]:
# PyPDFLoader - Document 객체 구조 확인
sample_doc = pypdf_docs[0]
print("📋 Document 객체 구조")
print("-"*40)
print(f"Type: {type(sample_doc)}")
print(f"\n메타데이터: {sample_doc.metadata}")
print(f"\n내용 (처음 300자):\n{sample_doc.page_content[:300]}")

📋 Document 객체 구조
----------------------------------------
Type: <class 'langchain_core.documents.base.Document'>

메타데이터: {'producer': 'ezPDF Builder Supreme', 'creator': 'PyPDF', 'creationdate': '2024-12-22T23:44:00+09:00', 'moddate': '2025-01-09T17:28:20+09:00', 'source': '2024년_원천징수의무자를_위한_연말정산_신고안내.pdf', 'total_pages': 426, 'page': 0, 'page_label': '1'}

내용 (처음 300자):
연말정산
신고안내
일 하나는 제대로 하는,
국민께 인정받는 국세청
2024. 12.
2024 원천징수의무자를 위한
맞춤형 안내
간소화 서비스
 일괄제공 서비스
발간등록번호
11-1210000-000072-10


---

## 3️⃣ PyMuPDFLoader로 문서 로드

### PyMuPDFLoader 특징
- **MuPDF 라이브러리** 기반 (C 바인딩)
- 텍스트 레이아웃 보존이 우수
- 더 빠른 처리 속도
- 이미지, 테이블 등 복잡한 구조 처리에 강점

In [ ]:
# PyMuPDFLoader 사용
print("="*60)
print("📄 PyMuPDFLoader 로드 시작")
print("="*60)

start_time = time.time()
pymupdf_loader = PyMuPDFLoader(PDF_PATH)
pymupdf_docs = pymupdf_loader.load()
pymupdf_time = time.time() - start_time

print(f"\n 로드 완료!")
print(f"총 페이지 수: {len(pymupdf_docs)}")
print(f"소요 시간: {pymupdf_time:.2f}초")

📄 PyMuPDFLoader 로드 시작

 로드 완료!
총 페이지 수: 426
소요 시간: 1.72초


In [ ]:
# PyMuPDFLoader - Document 객체 구조 확인
sample_doc = pymupdf_docs[0]
print("📋 Document 객체 구조")
print("-"*40)
print(f"Type: {type(sample_doc)}")
print(f"\n메타데이터: {sample_doc.metadata}")
print(f"\n내용 (처음 300자):\n{sample_doc.page_content[:300]}")

📋 Document 객체 구조
----------------------------------------
Type: <class 'langchain_core.documents.base.Document'>

메타데이터: {'producer': 'ezPDF Builder Supreme', 'creator': '', 'creationdate': '2024-12-22T23:44:00+09:00', 'source': '2024년_원천징수의무자를_위한_연말정산_신고안내.pdf', 'file_path': '2024년_원천징수의무자를_위한_연말정산_신고안내.pdf', 'total_pages': 426, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-01-09T17:28:20+09:00', 'trapped': '', 'modDate': "D:20250109172820+09'00'", 'creationDate': "D:20241222234400+09'00'", 'page': 0}

내용 (처음 300자):
연말정산
신고안내
일 하나는 제대로 하는,
국민께 인정받는 국세청
2024. 12.
2
0
2
4
 
원
천
징
수
의
무
자
를
 
위
한
맞춤형 안내
간소화 서비스
일괄제공 서비스
발
간
등
록
번
호
11-1210000-000072-10


---

## 4️⃣ Page 8 비교: 소득·세액공제 요건표

페이지 8에는 **표(Table)** 형태의 데이터가 포함되어 있습니다.  
두 로더가 표 구조를 어떻게 처리하는지 비교해봅시다.

In [ ]:
# Page 8 비교 (0-indexed이므로 index 7)
page_num = 8
idx = page_num - 1

print("="*80)
print(f"📑 Page {page_num} 비교: 소득·세액공제 요건표")
print("="*80)

📑 Page 8 비교: 소득·세액공제 요건표


In [ ]:
# PyPDFLoader - Page 8
print("\n" + "🔵 PyPDFLoader 결과".center(60, "-"))
print(pypdf_docs[idx].page_content)
print(f"\n📏 문자 수: {len(pypdf_docs[idx].page_content)}")


----------------------🔵 PyPDFLoader 결과----------------------
연말정산 소득·세액공제  
요건표
구 분
공 제 요 건
비 고
나이요건* 소득요건*
동거 요건
주민등록동거 일시퇴거 허용
기
본
공
제
본 인 × × ×
배 우 자 ×  ×
직계존속 60세 이상  △ 
(주거형편상 별거 허용) 1964.12.31. 이전
직계비속, 동거입양자 20세 이하  × 2004.  1.  1. 이후
장애인 직계비속의
장애인 배우자 ×  ×
형제자매 60세 이상
20세 이하    1964.12.31. 이전
2004.  1.  1. 이후
국민기초생활보장법에
의한 수급자 ×   
위탁아동 
추
가
공
제
장 애 인 기본공제대상자 중 장애인
경로우대 기본공제대상자 중 70세 이상인 자 1954.12.31.이전
부 녀 자 배우자가 없는 여성근로자로서 기본공제대상 부양가족이 있는 세대주 또는  
배우자가 있는 여성 근로자 (종합소득금액 3천만원 이하자)
한 부 모 배우자가 없는 자로서 부양자녀(20세 이하)가 있는자
연금보험료 공제 공적연금보험료의 근로자 본인 불입분만 공제 가능
* 나이요건    장애인의 경우 나이요건을 적용하지 않으며, 당해 과세기간 중 공제기준일이 해당하는 날이 있는 경우 적용
* 소득요건    연간 소득금액 100만원 이하(근로소득만 있는 경우 총급여 500만원 이하)
구 분
기본공제대상자의 요건 근로기간 지출한
비용만 공제 비 고
나이요건 소득요건
특별
소득공제
보 험 료 근로자 본인 부담분만 공제 가능 (건강·노인장기요양·고용보험료)
주택자금공제 - -  본인만 가능
그 밖의
소득공제
개인연금저축 근로자 본인 불입분만 공제 가능(배우자, 부양가족 불입분 제외)
주택마련저축 세대주인 근로자 본인 불입분만 공제 가능
신용카드 등 ×   형제자매 제외
자녀세액공제 (8세이상)   - 기본공제대상 자녀
(입양자·위탁아동·손자녀 포함)
2016.12.31. 
이전
연금계좌세액공제 근로자 본인 

In [ ]:
# PyMuPDFLoader - Page 8
print("\n" + "🟢 PyMuPDFLoader 결과".center(60, "-"))
print(pymupdf_docs[idx].page_content)
print(f"\n📏 문자 수: {len(pymupdf_docs[idx].page_content)}")


---------------------🟢 PyMuPDFLoader 결과---------------------
연말정산 소득·세액공제 
요건표
구 분
공 제 요 건
비 고
나이요건*
소득요건*
동거 요건
주민등록동거
일시퇴거 허용
기
본
공
제
본 인
×
×
×
배 우 자
×

×
직계존속
60세 이상

△ 
(주거형편상 별거 허용)
1964.12.31. 이전
직계비속, 동거입양자
20세 이하

×
2004.  1.  1. 이후
장애인 직계비속의
장애인 배우자
×

×
형제자매
60세 이상
20세 이하



1964.12.31. 이전
2004.  1.  1. 이후
국민기초생활보장법에
의한 수급자
×



위탁아동

추
가
공
제
장 애 인
기본공제대상자 중 장애인
경로우대
기본공제대상자 중 70세 이상인 자
1954.12.31.이전
부 녀 자
배우자가 없는 여성근로자로서 기본공제대상 부양가족이 있는 세대주 또는 
배우자가 있는 여성 근로자 (종합소득금액 3천만원 이하자)
한 부 모
배우자가 없는 자로서 부양자녀(20세 이하)가 있는자
연금보험료 공제
공적연금보험료의 근로자 본인 불입분만 공제 가능
* 나이요건    장애인의 경우 나이요건을 적용하지 않으며, 당해 과세기간 중 공제기준일이 해당하는 날이 있는 경우 적용
* 소득요건    연간 소득금액 100만원 이하(근로소득만 있는 경우 총급여 500만원 이하)
구 분
기본공제대상자의 요건
근로기간 지출한
비용만 공제
비 고
나이요건
소득요건
특별
소득공제
보 험 료
근로자 본인 부담분만 공제 가능 (건강·노인장기요양·고용보험료)
주택자금공제
-
-

본인만 가능
그 밖의
소득공제
개인연금저축
근로자 본인 불입분만 공제 가능(배우자, 부양가족 불입분 제외)
주택마련저축
세대주인 근로자 본인 불입분만 공제 가능
신용카드 등
×


형제자매 제외
자녀세액공제 (8세이상)


-
기본공제대상 자녀
(입양자·위탁아동·손자녀 포함)
2016.12.31.
이전
연금계좌세액공제
근로자 본인 불입분

---

## 5️⃣ Page 84 비교: 통상임금 계산 설명

페이지 84에는 **텍스트 + 박스 다이어그램**이 혼합되어 있습니다.  
복잡한 레이아웃을 어떻게 처리하는지 비교해봅시다.

In [ ]:
# Page 84 비교 (0-indexed이므로 index 83)
page_num = 84
idx = page_num - 1

print("="*80)
print(f"📑 Page {page_num} 비교: 통상임금 가산 급여")
print("="*80)

📑 Page 84 비교: 통상임금 가산 급여


In [ ]:
# PyPDFLoader - Page 84
print("\n" + "🔵 PyPDFLoader 결과".center(60, "-"))
print(pypdf_docs[idx].page_content)
print(f"\n📏 문자 수: {len(pypdf_docs[idx].page_content)}")


----------------------🔵 PyPDFLoader 결과----------------------
원천징수의무자를 위한 
2024년 연말정산 신고안내
68
   
참고
통상임금에 가산하여 지급받는 급여의 범위
근로기준법 제56조【연장·야간 및 휴일 근로】
사용자는 연장근로(제53조·제59조 및 제69조 단서에 따라 연장된 시간의 근로)와 휴일근로 및 야간근로(오후
10시부터 다음 날 오전 6시까지 사이의 근로)에 대하여는 통상임금의 100분의 50 (8시간을 초과한 휴일근로는 
100분의 100) 이상을 가산하여 지급하여야 한다.
○ 연장근로(통상임금의 50% 가산하여 지급) 
    
정상근무시간(8시간)
통상임금 10,000원
연장근무시간(4시간)
수당：5,000원(통상임금)
+ 2,500원(5,000×50%)
    ⇨ 연장근로수당 : 5,000원＋2,500원＝7,500원
○ 휴일근로(8시간 근로, 통상임금의 50% 가산하여 지급)
    수당：10,000원＋5,000원(10,000×50%)
연장근무시간(8시간)
    ⇨ 휴일근로수당: 10,000원＋5,000원＝15,000원
○ 야간(22：00~06：00)에 연장근로(4시간)하는 경우(통상임금의 50% 가산하여 지급)
    
정상근무시간(8시간)
통상임금 10,000원
연장·야간근로시간
수당：5,000원(통상임금)
+ 2,500원(연장근로)
+ 2,500원(야간근로)
    ⇨ 야간·연장근로시간 수당 : 5,000원＋2,500원＋2,500원＝10,000원
마. 비과세 식사대 등 (소법 §12 3호 러목)
  근로자가 사용자로부터 현물 식사 또는 금전으로 식사대를 제공받을 경우 비과세 되는 당해 
식사 또는 식사대는 다음의 요건에 해당하여야 한다.
1) 현물 식사
○ 근로자가 사내급식 또는 이와 유사한 방법으로 제공받는 식사 기타 음식물은 비과세한다. 
비과세되는 식사·기타 음식물이라 함은 사용자가 무상으로 제공하는 음식물로서 다음의 
요건에 해당하여야 한다.

📏 문자 수: 921


In [ ]:
# PyMuPDFLoader - Page 84
print("\n" + "🟢 PyMuPDFLoader 결과".center(60, "-"))
print(pymupdf_docs[idx].page_content)
print(f"\n📏 문자 수: {len(pymupdf_docs[idx].page_content)}")


---------------------🟢 PyMuPDFLoader 결과---------------------
원천징수의무자를 위한 
2024년 연말정산 신고안내
68
   
참고
통상임금에 가산하여 지급받는 급여의 범위
근로기준법 제56조【연장·야간 및 휴일 근로】
사용자는 연장근로(제53조·제59조 및 제69조 단서에 따라 연장된 시간의 근로)와 휴일근로 및 야간근로(오후
10시부터 다음 날 오전 6시까지 사이의 근로)에 대하여는 통상임금의 100분의 50 (8시간을 초과한 휴일근로는 
100분의 100) 이상을 가산하여 지급하여야 한다.
○ 연장근로(통상임금의 50% 가산하여 지급) 
    
정상근무시간(8시간)
통상임금 10,000원
연장근무시간(4시간)
수당：5,000원(통상임금)
+ 2,500원(5,000×50%)
    ⇨ 연장근로수당 : 5,000원＋2,500원＝7,500원
○ 휴일근로(8시간 근로, 통상임금의 50% 가산하여 지급)
    
수당：10,000원＋5,000원(10,000×50%)
연장근무시간(8시간)
    ⇨ 휴일근로수당: 10,000원＋5,000원＝15,000원
○ 야간(22：00~06：00)에 연장근로(4시간)하는 경우(통상임금의 50% 가산하여 지급)
    
정상근무시간(8시간)
통상임금 10,000원
연장·야간근로시간
수당：5,000원(통상임금)
+ 2,500원(연장근로)
+ 2,500원(야간근로)
    ⇨ 야간·연장근로시간 수당 : 5,000원＋2,500원＋2,500원＝10,000원
마. 비과세 식사대 등(소법 §12 3호 러목)
  근로자가 사용자로부터 현물 식사 또는 금전으로 식사대를 제공받을 경우 비과세 되는 당해 
식사 또는 식사대는 다음의 요건에 해당하여야 한다.
1) 현물 식사
○ 근로자가 사내급식 또는 이와 유사한 방법으로 제공받는 식사 기타 음식물은 비과세한다. 
비과세되는 식사·기타 음식물이라 함은 사용자가 무상으로 제공하는 음식물로서 다음의 
요건에 해당하여야 한다.

📏 문자 수: 921


---

## 6️⃣ 종합 비교 분석

In [ ]:
# 성능 및 품질 비교 요약
print("="*70)
print("📊 PyPDFLoader vs PyMuPDFLoader 비교 요약")
print("="*70)

print(f"\n{'항목':<20} {'PyPDFLoader':<20} {'PyMuPDFLoader':<20}")
print("-"*60)
print(f"{'로드 시간':<20} {pypdf_time:.2f}초{'':<14} {pymupdf_time:.2f}초")
print(f"{'속도 비율':<20} {'기준':<20} {pypdf_time/pymupdf_time:.1f}배 빠름")
print(f"{'Page 8 문자수':<20} {len(pypdf_docs[7].page_content):<20} {len(pymupdf_docs[7].page_content)}")
print(f"{'Page 84 문자수':<20} {len(pypdf_docs[83].page_content):<20} {len(pymupdf_docs[83].page_content)}")

📊 PyPDFLoader vs PyMuPDFLoader 비교 요약

항목                   PyPDFLoader          PyMuPDFLoader       
------------------------------------------------------------
로드 시간                61.35초               1.72초
속도 비율                기준                   35.7배 빠름
Page 8 문자수           1196                 1190
Page 84 문자수          921                  921


In [ ]:
# 텍스트 차이 분석
def compare_text_diff(text1, text2, label1="Text1", label2="Text2"):
    """두 텍스트의 차이점을 분석합니다."""
    # 공통 단어 / 고유 단어 분석
    words1 = set(text1.split())
    words2 = set(text2.split())

    common = words1 & words2
    only_in_1 = words1 - words2
    only_in_2 = words2 - words1

    print(f"\n📈 단어 분석")
    print(f"  - 공통 단어: {len(common)}개")
    print(f"  - {label1}에만 있는 단어: {len(only_in_1)}개")
    print(f"  - {label2}에만 있는 단어: {len(only_in_2)}개")

    # 일치율
    similarity = len(common) / max(len(words1), len(words2)) * 100
    print(f"  - 단어 일치율: {similarity:.1f}%")

    return only_in_1, only_in_2

print("\n" + "="*50)
print("Page 8 텍스트 차이 분석")
print("="*50)
only_pypdf, only_pymupdf = compare_text_diff(
    pypdf_docs[7].page_content,
    pymupdf_docs[7].page_content,
    "PyPDF", "PyMuPDF"
)


Page 8 텍스트 차이 분석

📈 단어 분석
  - 공통 단어: 160개
  - PyPDF에만 있는 단어: 2개
  - PyMuPDF에만 있는 단어: 2개
  - 단어 일치율: 98.8%


---

## 7️⃣ 메타데이터 비교

In [ ]:
# 메타데이터 비교
print("="*60)
print("📋 메타데이터 비교")
print("="*60)

print("\n🔵 PyPDFLoader 메타데이터:")
for key, value in pypdf_docs[0].metadata.items():
    print(f"   {key}: {value}")

print("\n🟢 PyMuPDFLoader 메타데이터:")
for key, value in pymupdf_docs[0].metadata.items():
    print(f"   {key}: {value}")

📋 메타데이터 비교

🔵 PyPDFLoader 메타데이터:
   producer: ezPDF Builder Supreme
   creator: PyPDF
   creationdate: 2024-12-22T23:44:00+09:00
   moddate: 2025-01-09T17:28:20+09:00
   source: 2024년_원천징수의무자를_위한_연말정산_신고안내.pdf
   total_pages: 426
   page: 0
   page_label: 1

🟢 PyMuPDFLoader 메타데이터:
   producer: ezPDF Builder Supreme
   creator: 
   creationdate: 2024-12-22T23:44:00+09:00
   source: 2024년_원천징수의무자를_위한_연말정산_신고안내.pdf
   file_path: 2024년_원천징수의무자를_위한_연말정산_신고안내.pdf
   total_pages: 426
   format: PDF 1.6
   title: 
   author: 
   subject: 
   keywords: 
   moddate: 2025-01-09T17:28:20+09:00
   trapped: 
   modDate: D:20250109172820+09'00'
   creationDate: D:20241222234400+09'00'
   page: 0


---

## 💡 결론 및 권장사항

### 어떤 로더를 선택해야 할까?

| 상황 | 권장 로더 | 이유 |
|------|----------|------|
| **대용량 PDF 처리** | PyMuPDFLoader | 속도가 빠름 |
| **표/테이블이 많은 문서** | PyMuPDFLoader | 레이아웃 보존 우수 |
| **단순 텍스트 문서** | PyPDFLoader | 가볍고 의존성 적음 |
| **한글 문서** | PyMuPDFLoader | 한글 인코딩 처리 안정적 |
| **설치 제약 환경** | PyPDFLoader | 순수 Python |

### 🎯 이 미션에서의 선택

연말정산 신고안내 문서는 **표, 법조문, 계산식** 등 복잡한 구조를 포함하고 있으므로,  
**PyMuPDFLoader**를 사용하는 것이 RAG 성능에 유리합니다.

---